Testing timeseries with photometry objects

In [1]:
import numpy as np
import astropy
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.timeseries import TimeSeries

In [2]:
ts1 = TimeSeries(time_start='2016-03-22T12:30:31',
                 time_delta=3 * u.s,
                 n_samples=5)
ts1

time
Time
2016-03-22T12:30:31.000
2016-03-22T12:30:34.000
2016-03-22T12:30:37.000
2016-03-22T12:30:40.000
2016-03-22T12:30:43.000


Plot flux against time series for a night

In [1]:
import os
import astropy
import numpy as np
from astropy.io import fits
from matplotlib import pyplot as plt
from astropy.visualization import ImageNormalize, LinearStretch, ZScaleInterval
from photutils.datasets import make_noise_image
from astropy.stats import sigma_clip
from scipy.stats import mode
from astropy.modeling import models, fitting
from astropy.table import Table
import photutils
from photutils.detection import DAOStarFinder
from astropy.stats import sigma_clipped_stats
from photutils.profiles import RadialProfile
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry, ApertureStats

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:


times = []
fluxes = []

for image in image_list: #Use the reduced science images here.
    
    radii = 15
    position = (612,612)
    
    science, header = fits.getdata(image, header=True) # read in the science image
    time = header.get('DATE-OBS') # Get the time from the image
   
    aps = CircularAperture(position, radii) # Make apertures

    phot_table = aperture_photometry(science, aps) # The last column of the table is flux

    annulus = CircularAnnulus((position), r_in=15, r_out=25) # make annuli

    ap_stats = ApertureStats(science, annulus) # get aperture stats
    
    background = []
    for i in range(0, np.size(ap_stats)):
        background.append(ap_stats[i].median)

    rows, cols = np.size(radii), np.size(background)
    sky_flux = [[0 for _ in range(cols)] for _ in range(rows)]
    #print(aps[0].r)
    for i in range(0, np.size(background)):
        for j in range (0, 1):
            aperture_area = aps[j].area_overlap(science)
            sky_flux[j][i] = (background[i] * aperture_area[1]) # sky flux (all apertures) , each array is one radius, each has n entries, n stars
    # then do background subtraction
    
    rows, cols = 1, np.size(background)
    sky_flux = [[0 for _ in range(cols)] for _ in range(rows)]
    #print(aps[0].r)
    for i in range(0, np.size(background)):
        for j in range (0, np.size(radii)):
            aperture_area = aps[j].area_overlap(science)
            sky_flux[j][i] = (background[i] * aperture_area[1]) # sky flux (all apertures) , each array is one radius, each has n entries, n stars
    # then do background subtraction
    for i in range(0, len(radii)):
        flux = phot_table[f'aperture_sum_{i}'] - sky_flux[i]
        
    times.append(time)
    fluxes.append(flux[0])

ValueError: 'r' must be a positive scalar

In [5]:
os.path.abspath("")

'/home/jovyan/arcsat_paper'

In [25]:
image_list = ['/home/jovyan/ccd_data/kelt-16-b-S001-R001-C084-r.fit']
times = []
fluxes = []

for image in image_list: # Use the reduced science images here.
    
    radius = 15 # Single radius value (not a list)
    position = (612, 612)
    
    science, header = fits.getdata(image, header=True) # Read the data
    
    time = header.get('DATE-OBS') # Get time of observation
    
    # Make aperture
    aperture = CircularAperture(position, r=radius)
    phot_table = aperture_photometry(science, aperture)
    # Make annulus
    annulus = CircularAnnulus(position, r_in=15, r_out=25)
    ap_stats = ApertureStats(science, annulus)
    
    # Get median background
    background = ap_stats.median
    
    # Calculate flux (background * area)
    aperture_area = aperture.area_overlap(science)
    sky_flux = background * aperture_area
    
    # Subtract background
    flux = phot_table['aperture_sum'] - sky_flux
    
    times.append(time) #Add to the time table
    fluxes.append(flux[0]) #Add to the flux table

In [30]:
#print(header)

In [28]:
times

['2018-08-23T09:00:21']

In [29]:
fluxes

[np.float64(2186.2671158937737)]

In [17]:
phot_table['aperture_sum'] - sky_flux

0.32636810721896836
